In [73]:
import numpy as np 
import pandas as pd

In [74]:
agreement = ['50', '50+', '67', '75', '100']

In [75]:
df = pd.read_csv('esg_bank_' + agreement[0] + '_agreement/esg_bank_' + agreement[0] + '_agreement.csv')
df['wc'] = df.apply(lambda x : len(x['sentence'].split()), axis = 1)

def length_label(word_count) :
    if word_count <= 15 :
        return 'short'
    else :
        if word_count <= 25 :
            return 'medium'
        else :
            if word_count <= 50 :
                return 'average'
            else :
                return 'long'
df['length_label'] = df.apply(lambda x : length_label(x['wc']), axis = 1)
df

,global_index,sentence,max_label,wc,length_label
0,1623,"(2) In clinical studies in over 600 subjects, ...",0,25,medium
1,4632,"(2) Represents 7,192,540 shares of common stoc...",0,36,average
2,2673,(6)| Includes expenses relating to the registr...,0,38,average
3,718,", banks are no longer prohibited from associat...",0,22,medium
4,1,","" ""Item 7. Management's Discussion and Analys...",0,203,long
...,...,...,...,...,...
4761,2934,•We offer competitive salary and benefit packa...,1,23,medium
4762,1424,•We provide inclusive family planning benefits...,1,28,average
4763,2379,•Wireless remote control that communicates wit...,0,39,average
4764,4176,•Women’s Innovation Network: Creates networkin...,1,14,short


A BIT OF STATISTICS

In [76]:
df.length_label.value_counts(normalize=True)

length_label
average    0.501259
long       0.230382
medium     0.209400
short      0.058959
Name: proportion, dtype: float64

In [77]:
df.max_label.value_counts(normalize=True)

max_label
0    0.652329
1    0.217373
2    0.130298
Name: proportion, dtype: float64

In [78]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Create a combined stratification column
df['stratify_col'] = (
    df['length_label'].astype(str) + "_" + df['max_label'].astype(str)
)

# Perform the 80-20 train-test split with stratification
train_data, test_data = train_test_split(
    df,
    test_size=0.2,
    stratify=df['stratify_col'],
    random_state=42
)

# Drop the helper stratify column (optional)
train_data = train_data.drop(columns=['stratify_col'])
test_data = test_data.drop(columns=['stratify_col'])

# Output the results
print("Train Data Distribution (Length Label):")
print(train_data['length_label'].value_counts(normalize=True))

print("\nTrain Data Distribution (Sentiment Label):")
print(train_data['max_label'].value_counts(normalize=True))

print("\nTest Data Distribution (Length Label):")
print(test_data['length_label'].value_counts(normalize=True))

print("\nTest Data Distribution (Sentiment Label):")
print(test_data['max_label'].value_counts(normalize=True))

Train Data Distribution (Length Label):
length_label
average    0.501312
long       0.230063
medium     0.209601
short      0.059024
Name: proportion, dtype: float64

Train Data Distribution (Sentiment Label):
max_label
0    0.652413
1    0.217471
2    0.130115
Name: proportion, dtype: float64

Test Data Distribution (Length Label):
length_label
average    0.501048
long       0.231656
medium     0.208595
short      0.058700
Name: proportion, dtype: float64

Test Data Distribution (Sentiment Label):
max_label
0    0.651992
1    0.216981
2    0.131027
Name: proportion, dtype: float64


In [79]:
train_data.to_csv('esg_bank_' + agreement[0] + '_agreement/training_set_with_metadata.csv', index=False)
train_data[['global_index', 'sentence', 'max_label']].to_csv('esg_bank_' + agreement[0] + '_agreement/training_set.csv', index=False)

In [80]:
test_data.to_csv('esg_bank_' + agreement[0] + '_agreement/test_set_with_metadata.csv', index=False)
test_data[['global_index', 'sentence', 'max_label']].to_csv('esg_bank_' + agreement[0] + '_agreement/test_set.csv', index=False)